In [1]:
import random
import re
from typing import List, Tuple
import numpy as np


Q-learning is model based (no prior knowlegde of the env)

based on Markov decision process

used in situation with finite actions, states and steps

works by exploring every action at every state and evaluates by assigning it a Q-Value

Q-table stores best action at any state (main brain)


In [2]:
grid_size = 4
start = (0, 0)
goal = (3, 3)
obstacle = [(1, 1), (2, 3), (3, 1)]
#
actions = [
    (-1, 0),
    (1, 0),
    (0, -1),
    (0, 1),  # up  # down  # left  # right
]


Ensures we're within grid boundaries, and didnt run into an opstacle

In [3]:
def is_valid_state(state: Tuple[int, int]) -> bool:
    return 0 <= state[0] < grid_size and 0 <= state[1] < grid_size and state not in obstacle

next state func to move from a given state toward a direction and checks if new state is valid


In [4]:
def get_next_state(state: Tuple[int, int], action: Tuple[int, int]) -> Tuple[int, int]:
    new_state = (state[0] + action[0], state[1] + action[1])
    return new_state if is_valid_state(new_state) else state

 ## Q-Learning params:
*   explr_rate: exploration 30% vs exploitation param 70%
*   lr: for learning rate, how much of teh new info is kept, lr = 0.3 -> incorporate 30% of new info and retain 70% of old info when
 updating q-value
*   disct_fct is the discount factor, gamma = 0.99 -> future estimated rewards are valuated at %99 of their actual value
immediate rewards are considered a little more



In [5]:
# Q-Learning params
explr_rate = 0.3

lr = 0.1

disct_fct = 0.99
episodes = 100000


- Rewards system: some feedback signal that inform the agent whether the actions are good or bad, so reward arriving at goal, and penalize hittig obstacles

- get reward formula: goal-> +100, obstacle or wall -> -10, for each step -> -1 (to find the best path)


In [6]:
def get_reward(state: Tuple[int, int], next_state: Tuple[int, int]) -> int:
    if next_state == goal:
        return 100
    #elif (next_state in obstacle) oindicesr (next_state == state):
    elif next_state == state:
        return -10
    else:
        return -1

- choose action determines whetehr the next move will be an exploratory move, or a conservative one picked from the q-table, this is the epsilon-greedy strategy

- it assured continous learning even when the agent finds a good policy returns a move (up, down...)


In [7]:
def choose_action(state: Tuple[int, int], q_table: np.ndarray) -> Tuple[int, int]:
    x, y = state
    if random.uniform(0, 1) < explr_rate:
        return random.choice(actions)
    else:
        return actions[np.argmax(q_table[x, y])]

- q value of each action at a given state is calculated using belman equation

- disct_fct * np.max(q_table[next_state]) is to factor in the future reward so best action is the one that maximizes both the immediate & future reward


In [8]:
def update_qtable(
    q_table: np.ndarray,
    state: Tuple[int, int],
    action: Tuple[int, int],
    next_state: Tuple[int, int],
    reward: int,
):
    action_index = actions.index(action)
    q_table[state][action_index] += lr * (
        reward + disct_fct * np.max(q_table[next_state]) - q_table[state][action_index]
    )

## training process
- 1 create a zeroed table same size as the grid, and start at (0, 0)
- 2 choose action based on current state and q_table and ger next_state
- 3 calculate reward + Q value
- 4 go to next state and repeat

In [14]:
def train_agent() -> np.ndarray:
    q_table = np.zeros((grid_size, grid_size, len(actions)))
#
    for _ in range(episodes):
        state = start
        steps = 0
        max_steps_per_episode = 100  # Prevent infinite loops

        while state != goal and steps < max_steps_per_episode:
            action = choose_action(state, q_table)
            next_state = get_next_state(state, action)
            reward = get_reward(state, next_state)
            update_qtable(q_table, state, action, next_state, reward)
            state = next_state
            steps += 1
#
    return q_table
#
q_table = train_agent()
print(q_table)

[[[ 79.29602988  90.19800998  79.29602988  90.19800998]
  [ 81.19800998  81.19800998  88.29602988  92.119202  ]
  [ 83.119202    94.0598      90.19800998  90.19800998]
  [ 81.19800888  92.11916561  92.119202    81.19800972]]

 [[ 88.29602988  92.119202    81.19800998  81.19800998]
  [  0.           0.           0.           0.        ]
  [ 92.119202    96.02        85.0598      92.119202  ]
  [ 90.19800994  83.119202    94.0598      83.119202  ]]

 [[ 90.19800998  90.19800998  83.119202    94.0598    ]
  [ 85.0598      85.0598      92.119202    96.02      ]
  [ 94.0598      98.          94.0598      87.02      ]
  [  0.           0.           0.           0.        ]]

 [[ 92.119202    81.19800998  81.19800998  81.19800998]
  [  0.           0.           0.           0.        ]
  [ 96.02        89.          89.         100.        ]
  [  0.           0.           0.           0.        ]]]


In [10]:
def visualize_q_table_as_grid(q_table: np.ndarray) -> None:
    action_symbols = ["^", ">", "v", "<"]
    print("\nQ-table Grid:")
    #
    header = (
        "   |"
        + "|".join(
            f"   ({i},{j})   " for i in range(grid_size) for j in range(grid_size)
        )
        + "|"
    )
    print(header)
    print("-" * len(header))
    #
    for action_idx, action_symbol in enumerate(action_symbols):
        row = f" {action_symbol} |"
        for i in range(grid_size):
            for j in range(grid_size):
                if (i, j) == goal:
                    cell = "   GOAL    "
                elif (i, j) in obstacle:
                    cell = " OBSTACLE  "
                else:
                    q_value = q_table[i, j, action_idx]
                    cell = f" {q_value:9.2f} "
                row += cell + "|"
        print(row)
        print("-" * len(header))
#
#
visualize_q_table_as_grid(q_table)



Q-table Grid:
   |   (0,0)   |   (0,1)   |   (0,2)   |   (0,3)   |   (1,0)   |   (1,1)   |   (1,2)   |   (1,3)   |   (2,0)   |   (2,1)   |   (2,2)   |   (2,3)   |   (3,0)   |   (3,1)   |   (3,2)   |   (3,3)   |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 ^ |     79.30 |     81.20 |     83.12 |     81.19 |     88.30 | OBSTACLE  |     92.12 |     90.20 |     90.20 |     85.06 |     94.06 | OBSTACLE  |     92.12 | OBSTACLE  |     96.02 |   GOAL    |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 > |     90.20 |     81.20 |     94.06 |     92.12 |     92.12 | OBSTACLE  |     96.02 |     83.12 |     90.20 |     85.06 |     98.00 | OBSTACLE  |     81.20 | OBSTACLE  |     89.00 |   GOAL    |


In [11]:
def visualize_policy(q_table: np.ndarray) -> None:
    """Visualize the learned policy as arrows"""
    action_symbols = ["↑", "↓", "←", "→"]
    print("\n=== Learned Policy ===")

    for i in range(grid_size):
        row = ""
        for j in range(grid_size):
            if (i, j) == goal:
                row += " G "
            elif (i, j) in obstacle:
                row += " X "
            else:
                best_action = np.argmax(q_table[i, j])
                row += f" {action_symbols[best_action]} "
        print(row)

visualize_policy(q_table)


=== Learned Policy ===
 ↓  →  ↓  ↓ 
 ↓  X  ↓  ← 
 →  →  ↓  X 
 ↑  X  →  G 


In [13]:
def test_agent(q_table: np.ndarray) -> None:
    """Test the trained agent"""
    print("\n=== Testing Trained Agent ===")
    state = start
    path = [state]
    steps = 0
    max_steps = 20

    while state != goal and steps < max_steps:
        action = choose_action(state, q_table)
        next_state = get_next_state(state, action)
        path.append(next_state)
        state = next_state
        steps += 1

    print(f"Path from {start} to {goal}:")
    for i, pos in enumerate(path):
        print(f"Step {i}: {pos}")
    print(f"Total steps: {len(path) - 1}")

    if state == goal:
        print("✓ Successfully reached goal!")
    else:
        print("✗ Failed to reach goal")


test_agent(q_table)


=== Testing Trained Agent ===
Path from (0, 0) to (3, 3):
Step 0: (0, 0)
Step 1: (1, 0)
Step 2: (2, 0)
Step 3: (2, 1)
Step 4: (2, 2)
Step 5: (3, 2)
Step 6: (3, 3)
Total steps: 6
✓ Successfully reached goal!
